In [1]:
%pip install langgraph google-cloud-aiplatform pinecone pydantic --quiet
!python -m pip install --upgrade google-genai --quiet
!python -m pip install dotenv --quiet


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from google import genai
import os

In [3]:
PROJECT_ID = "bdc-trainings"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "global")

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

In [4]:
if not client.vertexai:
    print("Using Gemini Developer API.")
elif client._api_client.project:
    print(
        f"Using Vertex AI with project: {client._api_client.project} in location: {client._api_client.location}"
    )
elif client._api_client.api_key:
    print(
        f"Using Vertex AI in express mode with API key: {client._api_client.api_key[:5]}...{client._api_client.api_key[-5:]}"
    )

Using Vertex AI with project: bdc-trainings in location: global


In [5]:
EMBEDDING_MODEL = "models/gemini-embedding-001"
LLM_MODEL = "gemini-2.5-flash"

In [6]:
import time
from dataclasses import dataclass
from typing import List, Dict, Any

# Pinecone (serverless)
from pinecone import Pinecone, ServerlessSpec
import logging


In [7]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise RuntimeError("PINECONE_API_KEY not found. Please set it in your environment.")
else:
  print("PINECONE API KEY found")

pc = Pinecone(api_key=PINECONE_API_KEY)

PINECONE API KEY found


In [8]:
print(PINECONE_API_KEY)

pcsk_6do2xF_TffEFjWU979gXVgaTpEWEv8tMFSFV7QgaRGEmdYa2gumrfkaVmiVodVDtbk24vC


In [9]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Change to DEBUG for more detailed logs
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

In [10]:
# Configure logging to both console and file
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("rag_workflow.log"),  # Logs to file
        logging.StreamHandler()                   # Logs to console
    ]
)

logger = logging.getLogger(__name__)

In [11]:
import json
import pinecone

from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL)

test_dim = len(embeddings.embed_query("dimension probe"))

/home/zadmin/Desktop/test/GAAI-B5-GCP/genai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
print(test_dim)

3072


In [13]:
INDEX_NAME = "makp" 
METRIC = "cosine"

# Create the index if it doesn't exist
existing = [idx["name"] for idx in pc.list_indexes()]
if INDEX_NAME not in existing:
    print(f"Creating index '{INDEX_NAME}' ...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=test_dim, #3072
        metric=METRIC,
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),)
    # optional: wait a moment for the index to be ready
    time.sleep(5)
else:
    print(f"Index '{INDEX_NAME}' already exists, reusing it.")

index = pc.Index(INDEX_NAME)
print(index.describe_index_stats())

Index 'makp' already exists, reusing it.
{'dimension': 3072,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 30}},
 'total_vector_count': 30,
 'vector_type': 'dense'}


In [14]:
# Load your dataset
with open("self_critique_loop_dataset.json") as f:
    kb_data = json.load(f)

In [15]:
print(kb_data)

[{'doc_id': 'KB001', 'question': 'What are best practices for debugging?', 'answer_snippet': "When addressing debugging, it's important to follow well-defined patterns...", 'source': 'debugging_guide.md', 'confidence_indicator': 'moderate', 'last_updated': '2024-01-10'}, {'doc_id': 'KB002', 'question': 'What are best practices for performance tuning?', 'answer_snippet': "When addressing performance tuning, it's important to follow well-defined patterns...", 'source': 'performance tuning_guide.md', 'confidence_indicator': 'moderate', 'last_updated': '2024-02-10'}, {'doc_id': 'KB003', 'question': 'What are best practices for caching?', 'answer_snippet': "When addressing caching, it's important to follow well-defined patterns...", 'source': 'caching_guide.md', 'confidence_indicator': 'moderate', 'last_updated': '2024-03-10'}, {'doc_id': 'KB004', 'question': 'What are best practices for asynchronous programming?', 'answer_snippet': "When addressing asynchronous programming, it's important 

In [16]:
# Extract texts
texts = [entry["answer_snippet"] for entry in kb_data]

# Get embeddings
embeddings_1 = embeddings.embed_documents(texts)

In [17]:
# Upsert
vectors = [{
    "id": item["doc_id"],
    "values": vector,
    "metadata": {
        "question": item["question"],
        "snippet": item["answer_snippet"],
        "source": item["source"]
    }
} for item, vector in zip(kb_data, embeddings_1)]

index.upsert(vectors)

print("KB Indexed into Pinecone")

KB Indexed into Pinecone


In [18]:
###testing
query = "What are best practices for performance tuning?"
k = 5

# Get the embedding vector for the query
query_vector = embeddings.embed_query(query)

# Query Pinecone index using the vector
results = index.query(
    vector=query_vector,
    top_k=k,
    include_metadata=True
)

In [19]:
results

{'matches': [{'id': 'KB002',
              'metadata': {'question': 'What are best practices for '
                                       'performance tuning?',
                           'snippet': 'When addressing performance tuning, '
                                      "it's important to follow well-defined "
                                      'patterns...',
                           'source': 'performance tuning_guide.md'},
              'score': 0.918594599,
              'values': []},
             {'id': 'KB022',
              'metadata': {'question': 'What are best practices for '
                                       'performance tuning?',
                           'snippet': 'When addressing performance tuning, '
                                      "it's important to follow well-defined "
                                      'patterns...',
                           'source': 'performance tuning_guide.md'},
              'score': 0.918594599,
              'values

In [20]:
# Retrieval Node
def retrieve_kb(question, top_k=5):
    query_vector = embeddings.embed_query(question)
    results = index.query(
    vector=query_vector,
    top_k=5,
    include_metadata=True
)
    return results["matches"]

In [21]:
##testing the retrieval node
query="What are best practices for asynchronous programming?"
retrieve_kb(query)
snippets=retrieve_kb(query)

In [22]:
snippets

[{'id': 'KB004',
  'metadata': {'question': 'What are best practices for asynchronous '
                           'programming?',
               'snippet': "When addressing asynchronous programming, it's "
                          'important to follow well-defined patterns...',
               'source': 'asynchronous programming_guide.md'},
  'score': 0.94732666,
  'values': []},
 {'id': 'KB014',
  'metadata': {'question': 'What are best practices for asynchronous '
                           'programming?',
               'snippet': "When addressing asynchronous programming, it's "
                          'important to follow well-defined patterns...',
               'source': 'asynchronous programming_guide.md'},
  'score': 0.94732666,
  'values': []},
 {'id': 'KB024',
  'metadata': {'question': 'What are best practices for asynchronous '
                           'programming?',
               'snippet': "When addressing asynchronous programming, it's "
                         

In [23]:
# Generation Node
def generate_answer(question, snippets):
    context = "\n".join([f"[{s['id']}] {s['metadata']['snippet']}" for s in snippets])
    prompt = f"""Use the context below to answer the question:
Context:
{context}
Question: {question}
Answer with citations in the format [KBxxx]."""

    return client.models.generate_content(contents=prompt, model=LLM_MODEL).text.strip()

In [24]:
answer=generate_answer(query,snippets=snippets)
answer

2025-10-05 22:32:22,895 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-05 22:32:26,277 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


"When addressing asynchronous programming, it's important to follow well-defined patterns [KB004], [KB014], [KB024]."

In [25]:
#!gcloud auth application-default login

In [26]:
# Critique Node
def critique_answer(question, snippets, answer):
    context = "\n".join([f"- {s['metadata']['snippet']}" for s in snippets])
    prompt = f"""Given the context and an answer, determine if the answer is COMPLETE or needs REFINEMENT.
Context:
{context}

Question: {question}

Answer:
{answer}

Respond in one of the following formats:
- COMPLETE
- REFINE: <list missing keywords or ideas>
"""
    return client.models.generate_content(contents=prompt, model=LLM_MODEL).text.strip()

In [27]:
critique_answer(query, snippets, answer)

2025-10-05 22:32:31,567 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-05 22:32:34,464 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


'COMPLETE'

In [28]:
# Refinement Node
def refine_answer(question, prev_snippets, missing_keywords):
    new_query = f"{question} {missing_keywords}"
    extra_snippet = retrieve_kb(new_query, top_k=1)
    combined = prev_snippets + extra_snippet
    return generate_answer(question, combined)

In [29]:
refine_answer(query, prev_snippets=snippets, missing_keywords='REFINE: performance tuning')

2025-10-05 22:32:37,823 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-05 22:32:39,877 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"


"It's important to follow well-defined patterns [KB004, KB014, KB024]."

In [30]:
# Build LangGraph Workflow 


from typing import TypedDict
from langgraph.graph import StateGraph


# Define the schema for the graph state
class RAGState(TypedDict):
    question: str
    snippets: list
    answer: str
    critique: str
    final_answer: str


def build_rag_graph():
    graph = StateGraph(RAGState)
  
    def retrieve(state):
        question = state["question"]
        logger.info(f"Retrieving snippets for question: {question}")
        snippets = retrieve_kb(question)
        logger.info(f"[Retriever] Retrieved {len(snippets)} snippets")

        return {"question": question, "snippets": snippets}
    
    def generate(state):
        logger.info(f"[GenerateAnswer] Generating answer for: {state['question']}")
        answer = generate_answer(state["question"], state["snippets"])
        logger.info(f"[GenerateAnswer] Generated answer: {answer[:100]}...")
        return {**state, "answer": answer}

    def critique(state):
        logger.info(f"[Critique] Critiquing answer...")
        result = critique_answer(state["question"], state["snippets"], state["answer"])
        logger.info(f"[Critique] Critique result: {result}")
        return {**state, "critique": result}    
    
    def needs_refine(state):
        decision = "REFINE" in state["critique"]
        logger.info(f"[CheckCritique] Needs refinement? {decision}")
        return decision

    
    def refine(state):
        logger.info(f"[Refine] Refining answer...")
        missing = state["critique"].split("REFINE:")[-1].strip()
        final_answer = refine_answer(state["question"], state["snippets"], missing)
        logger.info(f"[Refine] Final refined answer: {final_answer[:100]}...")
        return {**state, "final_answer": final_answer}
    
    def end_node(state):
        logger.info(f"[END] Final state reached.")
        return state  # Just passes through

    graph.add_node("Retriever", retrieve)
    graph.add_node("GenerateAnswer", generate)
    graph.add_node("Critique", critique)
    graph.add_node("Refine", refine)
    graph.add_node("END", end_node)

    # Add condition
    graph.add_conditional_edges("Critique", needs_refine, {
        True: "Refine",
        False: "END"
    })

   
# Set entry and edges
    graph.set_entry_point("Retriever")
    graph.add_edge("Retriever", "GenerateAnswer")
    graph.add_edge("GenerateAnswer", "Critique")
    graph.add_edge("Refine", "END")

    return graph.compile()    

In [31]:
graph= build_rag_graph()

question = input("Enter your question: ")
logger.info(f"[Main] Starting RAG workflow for question: {question}")

result = graph.invoke({"question": question})

logger.info(f"[Main] Workflow completed. Final result: {result.get('final_answer', result.get('answer'))}")
print(result.get("final_answer", result.get("answer")))

2025-10-05 22:32:54,746 - __main__ - INFO - [Main] Starting RAG workflow for question: How do I version my APIs?
2025-10-05 22:32:54,750 - __main__ - INFO - Retrieving snippets for question: How do I version my APIs?
2025-10-05 22:32:55,743 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-05 22:32:55,746 - __main__ - INFO - [GenerateAnswer] Generating answer for: How do I version my APIs?
2025-10-05 22:32:55,748 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-05 22:32:57,933 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-05 22:32:57,935 - __main__ - INFO - [GenerateAnswer] Generated answer: When addressing API versioning, it's important to follow well-defined patterns [KB005, KB015, KB025]...
2025-10-05 22:32:57,936 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-05 22:32:57,93

When addressing API versioning, it's important to follow well-defined patterns [KB025, KB015, KB005].


In [32]:
print(result.get("final_answer", result.get("answer")))

When addressing API versioning, it's important to follow well-defined patterns [KB025, KB015, KB005].


In [33]:
graph= build_rag_graph()

question=input("Enter your question: ")

result= graph.invoke({"question": question})


print(result.get("final_answer", result["answer"]))

2025-10-05 22:33:39,357 - __main__ - INFO - Retrieving snippets for question: What are performance tuning tips?
2025-10-05 22:33:40,337 - __main__ - INFO - [Retriever] Retrieved 5 snippets
2025-10-05 22:33:40,340 - __main__ - INFO - [GenerateAnswer] Generating answer for: What are performance tuning tips?
2025-10-05 22:33:40,340 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-05 22:33:43,263 - httpx - INFO - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/bdc-trainings/locations/global/publishers/google/models/gemini-2.5-flash:generateContent "HTTP/1.1 200 OK"
2025-10-05 22:33:43,265 - __main__ - INFO - [GenerateAnswer] Generated answer: It's important to follow well-defined patterns when addressing performance tuning [KB002, KB022, KB0...
2025-10-05 22:33:43,267 - __main__ - INFO - [Critique] Critiquing answer...
2025-10-05 22:33:43,267 - google_genai.models - INFO - AFC is enabled with max remote calls: 10.
2025-10-05 22:33:45,32

It's important to follow well-defined patterns when addressing performance tuning [KB002, KB022, KB012].
